# PART 1 - READ IN THE DATAFRAMES

# Direct Investment Current Cost Adjustments

This is an excerpt of a longer script that calculates (A) the current cost markup to the direct investment equity position and (B) the current cost adjustment to direct investment equity income.

(A) involves revaluing the following from historical cost to current cost:
1. structures & equipment (fixed assets)
2. inventories
3. land

(B) involves computing the consumption of fixed capital:
1. economic depreciation at current cost for structures & equipment (fixed assets)


For A.1, industry-by-year net fixed assets (obtained using entity-level direct investment data from BEA's DI surveys) are revalued to current cost from historical cost using ratios derived from BEA's Fixed Assets Accounts. For B.1, depreciation is computed using depreciation rates implied by the Fixed Assets Accounts. For both, since the necessary values are published, the BEA API can be used to retrieve them (rather than downloading excel files on our website).

To install *beaapi*, visit https://github.com/us-bea/beaapi/releases and download the wheel (.whl) file. Then, open your Anaconda terminal and run the following code (if a new version has been released, update the file name):
```
    pip install Downloads\beaapi-0.0.2-py3-none-any.whl
```

You will also need an API key. To register, visit https://apps.bea.gov/api/signup/ and fill out the registration form on the right hand side of the page. The API key will be emailed to you.

## I. Preliminaries

Enter your BEA API key:

In [1]:
beakey = '12345678-2222-3333-4444-000000000099'

Not all years of available data from the Fixed Assets Accounts are used for these calculations. They begin with 1995, and end with the most recent year of data available on MNEs from the annual DI surveys. Below, the desired start and end years are assigned to variables:

In [2]:
yearfirst = 1995
yearlast = 2020


Import packages needed for this code:

In [3]:
import numpy as np
import pandas as pd
import beaapi

## II. Import Fixed Assets Data

The *beaapi* ```get_data_set_list``` method returns the names and descriptions of the available table series:

In [4]:
beaapi.get_data_set_list(beakey)

DatasetName                                 DatasetDescription
0                      NIPA                               Standard NIPA tables
1        NIUnderlyingDetail               Standard NI underlying detail tables
2                       MNE                          Multinational Enterprises
3               FixedAssets                       Standard Fixed Assets tables
4                       ITA                International Transactions Accounts
5                       IIP                  International Investment Position
6               InputOutput                                  Input-Output Data
7             IntlServTrade                       International Services Trade
8               IntlServSTA  International Services Supplied Through Affili...
9             GDPbyIndustry                                    GDP by Industry
10                 Regional                                 Regional data sets
11  UnderlyingGDPbyIndustry                         Underlying GDP by Industry
12       APIDatasetMetaData                  Metadata about other API datasets

The ```get_parameter_list``` method returns the parameters for which we will need to pass arguments when retrieving the data:

In [5]:
beaapi.get_parameter_list(beakey, 'FixedAssets')

ParameterName ParameterDataType  \
0     TableName            string   
1          Year           integer   

                              ParameterDescription  ParameterIsRequiredFlag  \
0                  The new Fixed Assets identifier                        1   
1  List of year(s) of data to retrieve (X for All)                        1   

  ParameterDefaultValue  MultipleAcceptedFlag AllValue  
0                                           0           
1                                           1        X

The first parameter is **TableName**. Tables are obtained from BEA API using their table names (IDs), which can be found [here](https://apps.bea.gov/api/_pdf/bea_web_service_api_user_guide.pdf). Fixed Assets table names follow the naming convention ```FAAt###X```.

In [6]:
# A list of the table names
pftable_names = ['FAAt301E',
             'FAAt301S',
             'FAAt303E',
             'FAAt303S',
             'FAAt304E',
             'FAAt304S']
# same thing for residential FA tables:
rftable_names = ['FAAt501',
             'FAAt503',
             'FAAt504']

In [7]:
# The column names for each table
pfcol_names = ['ccposequip',
             'ccposstruct',
             'hcposequip',
             'hcposstruct',
             'ccdeprequip',
             'ccdeprstruct']
# same thing for residential FA tables:
rfcol_names = ['ccposres',
             'hcposres',
             'ccdeprres']


In [8]:
# Create 2 python dictionaires that map private FA table numbers to the descriptive col names that will be used later on to rename the "DataValue" col in each table:
# 
pfatables = dict(zip(pftable_names, pfcol_names))
# same thing for residential FA tables:
rfatables = dict(zip(rftable_names, rfcol_names))

In [9]:
# Check the dictionaries
pfatables, rfatables

({'FAAt301E': 'ccposequip',
  'FAAt301S': 'ccposstruct',
  'FAAt303E': 'hcposequip',
  'FAAt303S': 'hcposstruct',
  'FAAt304E': 'ccdeprequip',
  'FAAt304S': 'ccdeprstruct'},
 {'FAAt501': 'ccposres', 'FAAt503': 'hcposres', 'FAAt504': 'ccdeprres'})

**Year** is the other parameter. Use ```Year='X'``` as the argument for all years, ```Year='YYYY'``` for a single year, or ```Year='YYYY, YYYY, YYYY, ...'``` for multiple. Notice that, for multiple years, the argument is not a list of strings, but rather a flat string with commas between years. 

For many years it may be tedious to type them all manually, so you may wish to convert a list of numeric years to the flat string:

In [10]:
# create a list of numeric years using the variables defined in the previous section:
yearlist = [str(year) for year in range(yearfirst, yearlast+1)] 
yearstr = ', '.join(yearlist)

# Verify the string of years
print(type(yearstr), yearstr)

<class 'str'> 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020


# PART 2 - READ IN THE DATAFRAMES

To retrieve data using the BEA API, use the ```get_data``` method. For the Fixed Assets tables, where the parameters are **TableName** and **Year**, with **TableName** taking the form ```FAAt###X```, the code is as follows:
```
beaapi.get_data(beakey, 'FixedAssets', TableName=f'FAAt{###X}', Year='YYYY')
```

Import private fixed assets by industry tables and save each to a list of dataframes.

Do the same thing for residential fixed assets tables and save append a new list list of dataframes.

In [11]:
# These are the columns to store
cols_to_read = ['LineNumber', 'LineDescription', 'TimePeriod', 'DataValue']

fatables1 = [beaapi.get_data(beakey, 'FixedAssets', TableName=tab, Year=yearstr)[cols_to_read] for tab in pfatables]

fatables2 = [beaapi.get_data(beakey, 'FixedAssets', TableName=tab, Year=yearstr)[cols_to_read] for tab in rfatables]


The residential fixed asset tables need additional wrangling.

We want to have only the rows that have `Private` in the `LineDescription` column. 

Replace any `2` with a `1` in the `LineNumber` column for consistency with the fixed asset tables.


In [12]:
for table in fatables2:
    table = table[table['LineDescription'] == 'Private']
    table.replace({'LineNumber':{2:1}}, inplace=True)


c:\Users\student\AppData\Local\Programs\Python\Python37-32\lib\site-packages\pandas\core\indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)



Finally, concatenate into a single list the fixed asset datafromes followed by the residential dataframes.

Verify the 9 dataframes in the list

In [13]:
fatables = fatables1 + fatables2

print('table count', len(fatables))
for table in fatables:
    print(table.shape, table.head(1))

table count 9
(2496, 4)    LineNumber       LineDescription TimePeriod  DataValue
0           1  Private fixed assets       1995    2994776
(2496, 4)    LineNumber       LineDescription TimePeriod  DataValue
0           1  Private fixed assets       1995   12105745
(2496, 4)    LineNumber       LineDescription TimePeriod  DataValue
0           1  Private fixed assets       1995    2535898
(2496, 4)    LineNumber       LineDescription TimePeriod  DataValue
0           1  Private fixed assets       1995    6454856
(2496, 4)    LineNumber       LineDescription TimePeriod  DataValue
0           1  Private fixed assets       1995     398516
(2496, 4)    LineNumber       LineDescription TimePeriod  DataValue
0           1  Private fixed assets       1995     295031
(312, 4)    LineNumber           LineDescription TimePeriod  DataValue
0           1  Residential fixed assets       1995    7406620
(182, 4)    LineNumber LineDescription TimePeriod  DataValue
0           2         Private       

Each table has a **DataValue** column, though the value represented in it is different in each table. Using the column names (values) from the two dictionaries, create 
a list of these strings.

In [14]:
values = list(pfatables.values()) + list(rfatables.values())
values

['ccposequip',
 'ccposstruct',
 'hcposequip',
 'hcposstruct',
 'ccdeprequip',
 'ccdeprstruct',
 'ccposres',
 'hcposres',
 'ccdeprres']

The list of column names is in the same order as the DataFrames of the fixed asset tables.

Use a loop and numeric indexing to rename the `DataValue` column to it corresponding name.

For example, the first DataFrame renames `DataValue` to `ccposquip`.  The second DataFrame renames `DataValue` to `ccposstruct`

Finally, the ninth DataFrame renames `DataValue` to `ccdeprres`

In [15]:
for indx, table in enumerate(fatables):
    table.rename({'DataValue':values[indx]}, axis=1, inplace=True)

for table in fatables:
    print(table.head(1))

   LineNumber       LineDescription TimePeriod  ccposequip
0           1  Private fixed assets       1995     2994776
   LineNumber       LineDescription TimePeriod  ccposstruct
0           1  Private fixed assets       1995     12105745
   LineNumber       LineDescription TimePeriod  hcposequip
0           1  Private fixed assets       1995     2535898
   LineNumber       LineDescription TimePeriod  hcposstruct
0           1  Private fixed assets       1995      6454856
   LineNumber       LineDescription TimePeriod  ccdeprequip
0           1  Private fixed assets       1995       398516
   LineNumber       LineDescription TimePeriod  ccdeprstruct
0           1  Private fixed assets       1995        295031
   LineNumber           LineDescription TimePeriod  ccposres
0           1  Residential fixed assets       1995   7406620
   LineNumber LineDescription TimePeriod  hcposres
0           2         Private       1995   3723818
   LineNumber           LineDescription TimePeriod  ccdepr

Now that there are unique column names for the data values in each table, they can be merged into a single dataframe.



For the private fixed assets tables, merge them on these columns:
    'LineNumber', 'TimePeriod', 'LineDescription'

Assuming `fa` is the DataFrame to hold all results and `table` is the next DataFrame, do the merges similar to:

   `fa = pd.merge(fa, table, on=['LineNumber', 'TimePeriod', 'LineDescription'])`

In [16]:
num_pfatables = len(pfatables)
num_rfatables = len(rfatables)
fa = fatables[0]

for table in fatables[1:num_pfatables]:
    fa = pd.merge(fa, table, on=['LineNumber', 'TimePeriod', 'LineDescription']) 


For the residential fixed assets tables, a different merge is required.  The `LineDescription` column is dropped
and a `left` merge is required.

Assuming `fa` is the merged dataframe of private fixed asset tables, do the remaining 3 merges similar to:

   `fa = pd.merge(fa, table.drop(['LineDescription'], axis=1), on=['LineNumber', 'TimePeriod'], how='left')`

The residential fixed assets data were obtained only for the "Private" aggregate line; fill null values for the other lines with zero.

Verify the new dataframe

In [17]:
for table in fatables[num_pfatables:]:
    fa = pd.merge(fa, table.drop(['LineDescription'], axis=1), on=['LineNumber', 'TimePeriod'], how='left')   

fa.fillna(0, inplace=True)

fa.shape, fa.head(), fa.tail()

((2496, 12),
    LineNumber       LineDescription TimePeriod  ccposequip  ccposstruct  \
 0           1  Private fixed assets       1995     2994776     12105745   
 1           1  Private fixed assets       1996     3140588     12714845   
 2           1  Private fixed assets       1997     3271617     13438018   
 3           1  Private fixed assets       1998     3421541     14271166   
 4           1  Private fixed assets       1999     3616329     15203793   
 
    hcposequip  hcposstruct  ccdeprequip  ccdeprstruct   ccposres  hcposres  \
 0     2535898      6454856       398516        295031  7406620.0       0.0   
 1     2720384      6841577       419640        308620  7801886.0       0.0   
 2     2917240      7258084       440883        326273  8223911.0       0.0   
 3     3129521      7727779       461388        348201  8753459.0       0.0   
 4     3358112      8231945       488501        369742  9395249.0       0.0   
 
    ccdeprres  
 0   169863.0  
 1   178207.0  
 2   

# PART 3 - Calculating new columns

## III. Perform Calculations

First, calculate the current cost (CC) position to historical cost (HC) position ratios
There are three ratios obtained from these columns

    overall ratio: (ccposequip + ccposstruct) / (hcposequip + hcpossruct)
    
    nonresidential ratio: (ccposequip + ccposstruct - ccposres) / (hcposequip + hcposstruct - hcposres)
    
    residential ratio: ccposres / hcposres
    
Add a new column for each ratio

In [18]:
fa['cchcratio_fa'] = (fa['ccposequip'] + fa['ccposstruct']) / (fa['hcposequip'] + fa['hcposstruct'])
fa['cchcratio_fa_nonres'] = (fa['ccposequip'] + fa['ccposstruct'] - fa['ccposres']) / (fa['hcposequip'] + fa['hcposstruct'] - fa['hcposres'])
fa['cchcratio_fa_res'] = fa['ccposres'] / fa['hcposres']

fa.head(), fa.tail()

(   LineNumber       LineDescription TimePeriod  ccposequip  ccposstruct  \
 0           1  Private fixed assets       1995     2994776     12105745   
 1           1  Private fixed assets       1996     3140588     12714845   
 2           1  Private fixed assets       1997     3271617     13438018   
 3           1  Private fixed assets       1998     3421541     14271166   
 4           1  Private fixed assets       1999     3616329     15203793   
 
    hcposequip  hcposstruct  ccdeprequip  ccdeprstruct   ccposres  hcposres  \
 0     2535898      6454856       398516        295031  7406620.0       0.0   
 1     2720384      6841577       419640        308620  7801886.0       0.0   
 2     2917240      7258084       440883        326273  8223911.0       0.0   
 3     3129521      7727779       461388        348201  8753459.0       0.0   
 4     3358112      8231945       488501        369742  9395249.0       0.0   
 
    ccdeprres  cchcratio_fa  cchcratio_fa_nonres  cchcratio_fa_res

Next, calculate the depreciation rate (overall, nonresidential, and residential) for each line and year.

The depreciation rate calculate requires the lag of the CC positions `ccposequip`, `ccposstruct`, and `ccposres` for each line and year.

To add a new column of the lag, use this as a model:
    
    `fa['ccposequip_lag'] = fa.groupby('LineNumber')['ccposequip'].shift(1)`

Begin by sorting the dataframe by `LineNumber` and `TimePeriod`.  Then add the 3 lag columns.

In [19]:
fa.sort_values(['LineNumber', 'TimePeriod'], inplace=True)

fa['ccposequip_lag'] = fa.groupby('LineNumber')['ccposequip'].shift(1)
fa['ccposstruct_lag'] = fa.sort_values(['LineNumber', 'TimePeriod']).groupby('LineNumber')['ccposstruct'].shift(1)
fa['ccposres_lag'] = fa.sort_values(['LineNumber', 'TimePeriod']).groupby('LineNumber')['ccposres'].shift(1)

fa.head()

LineNumber       LineDescription TimePeriod  ccposequip  ccposstruct  \
0           1  Private fixed assets       1995     2994776     12105745   
1           1  Private fixed assets       1996     3140588     12714845   
2           1  Private fixed assets       1997     3271617     13438018   
3           1  Private fixed assets       1998     3421541     14271166   
4           1  Private fixed assets       1999     3616329     15203793   

   hcposequip  hcposstruct  ccdeprequip  ccdeprstruct   ccposres  hcposres  \
0     2535898      6454856       398516        295031  7406620.0       0.0   
1     2720384      6841577       419640        308620  7801886.0       0.0   
2     2917240      7258084       440883        326273  8223911.0       0.0   
3     3129521      7727779       461388        348201  8753459.0       0.0   
4     3358112      8231945       488501        369742  9395249.0       0.0   

   ccdeprres  cchcratio_fa  cchcratio_fa_nonres  cchcratio_fa_res  \
0   169863.0      1.679561             0.855757               inf   
1   178207.0      1.658178             0.842248               inf   
2   187511.0      1.642172             0.833951               inf   
3   199298.0      1.629568             0.823340               inf   
4   214457.0      1.623816             0.813186               inf   

   ccposequip_lag  ccposstruct_lag  ccposres_lag  
0             NaN              NaN           NaN  
1       2994776.0       12105745.0     7406620.0  
2       3140588.0       12714845.0     7801886.0  
3       3271617.0       13438018.0     8223911.0  
4       3421541.0       14271166.0     8753459.0

Calculate the three depreciation rates (overall, nonresidential, and residential) for each line and year.

        overall: (cddeprequip + ccdeprstruct) / 0.5 * (ccposequip + ccposstruct + ccposequip_lag + ccposstruct_lag)
        
        nonresidential: (ccdeprequip + ccdeprstruct - ccdeprres) / 
                         (0.5 * ((ccposequip + ccposstruct - ccposequip_lag) + (ccposequip_lag + ccpossstruct_lag - ccposres_lag)))
                         
        residential: ccdepres / (0.5 * (ccposres + ccposres_lag))

Add a new column for each depreciation rate

In [20]:
fa['deprrate_fa'] = (fa['ccdeprequip'] + fa['ccdeprstruct']) / (0.5 * ((fa['ccposequip'] + fa['ccposstruct']) + 
                                                                       (fa['ccposequip_lag'] + fa['ccposstruct_lag'])))
fa['deprrate_fa_nonres'] = (fa['ccdeprequip'] + fa['ccdeprstruct'] - fa['ccdeprres']) / (0.5 * ((fa['ccposequip'] + fa['ccposstruct'] - fa['ccposres']) + 
                                                                                                (fa['ccposequip_lag'] + fa['ccposstruct_lag'] - fa['ccposres_lag'])))
fa['deprrate_fa_res'] = fa['ccdeprres'] / (0.5 * (fa['ccposres'] + 
                                                  fa['ccposres_lag']))

fa.head()

LineNumber       LineDescription TimePeriod  ccposequip  ccposstruct  \
0           1  Private fixed assets       1995     2994776     12105745   
1           1  Private fixed assets       1996     3140588     12714845   
2           1  Private fixed assets       1997     3271617     13438018   
3           1  Private fixed assets       1998     3421541     14271166   
4           1  Private fixed assets       1999     3616329     15203793   

   hcposequip  hcposstruct  ccdeprequip  ccdeprstruct   ccposres  ...  \
0     2535898      6454856       398516        295031  7406620.0  ...   
1     2720384      6841577       419640        308620  7801886.0  ...   
2     2917240      7258084       440883        326273  8223911.0  ...   
3     3129521      7727779       461388        348201  8753459.0  ...   
4     3358112      8231945       488501        369742  9395249.0  ...   

   ccdeprres  cchcratio_fa  cchcratio_fa_nonres  cchcratio_fa_res  \
0   169863.0      1.679561             0.855757               inf   
1   178207.0      1.658178             0.842248               inf   
2   187511.0      1.642172             0.833951               inf   
3   199298.0      1.629568             0.823340               inf   
4   214457.0      1.623816             0.813186               inf   

   ccposequip_lag  ccposstruct_lag  ccposres_lag  deprrate_fa  \
0             NaN              NaN           NaN          NaN   
1       2994776.0       12105745.0     7406620.0     0.047051   
2       3140588.0       12714845.0     7801886.0     0.047115   
3       3271617.0       13438018.0     8223911.0     0.047066   
4       3421541.0       14271166.0     8753459.0     0.047010   

   deprrate_fa_nonres  deprrate_fa_res  
0                 NaN              NaN  
1            0.069859         0.023435  
2            0.070093         0.023401  
3            0.070048         0.023478  
4            0.070113         0.023633  

[5 rows x 21 columns]

Finally, only the `LineNumber`, `LineDescription`, `TimePeriod` and the calculated CC/HC ratios and depreciation rate columns for each line and year are used later on, so the remaining columns can be dropped:

In [21]:
fa.drop(['ccposequip', 'ccposstruct', 'hcposequip', 'hcposstruct', 'ccdeprequip', 'ccdeprstruct', 
         'ccposres', 'hcposres', 'ccdeprres',
         'ccposequip_lag', 'ccposstruct_lag', 'ccposres_lag'], axis=1, inplace=True)

fa

LineNumber                    LineDescription TimePeriod  cchcratio_fa  \
0              1               Private fixed assets       1995      1.679561   
1              1               Private fixed assets       1996      1.658178   
2              1               Private fixed assets       1997      1.642172   
3              1               Private fixed assets       1998      1.629568   
4              1               Private fixed assets       1999      1.623816   
...          ...                                ...        ...           ...   
2491          96  Other services, except government       2016      1.787142   
2492          96  Other services, except government       2017      1.808258   
2493          96  Other services, except government       2018      1.859758   
2494          96  Other services, except government       2019      1.884819   
2495          96  Other services, except government       2020      1.899064   

      cchcratio_fa_nonres  cchcratio_fa_res  deprrate_fa  deprrate_fa_nonres  \
0                0.855757               inf          NaN                 NaN   
1                0.842248               inf     0.047051            0.069859   
2                0.833951               inf     0.047115            0.070093   
3                0.823340               inf     0.047066            0.070048   
4                0.813186               inf     0.047010            0.070113   
...                   ...               ...          ...                 ...   
2491             1.787142               NaN     0.043699            0.043699   
2492             1.808258               NaN     0.043581            0.043581   
2493             1.859758               NaN     0.043293            0.043293   
2494             1.884819               NaN     0.042991            0.042991   
2495             1.899064               NaN     0.042822            0.042822   

      deprrate_fa_res  
0                 NaN  
1            0.023435  
2            0.023401  
3            0.023478  
4            0.023633  
...               ...  
2491              NaN  
2492              NaN  
2493              NaN  
2494              NaN  
2495              NaN  

[2496 rows x 9 columns]

The CC/HC ratios are used later to revalue industry-level net fixed assets from historical cost to current cost (an element in the calculation of the CC markup to DI equity position), and the depreciation rate to compute industry-level economic depreciation at current cost (an element in the calculation of the CC adjustment to DI equity income).